<a href="https://colab.research.google.com/github/sanjanabayya30/Generative_AI_2025/blob/main/GENAI_ProjectCode_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries (Colab-specific installation)
!pip install -q gradio huggingface_hub

# Import necessary libraries
import gradio as gr
from huggingface_hub import InferenceClient, login
import getpass

# Authenticate with Hugging Face
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(token=HF_TOKEN)

# Initialize Hugging Face Inference Client
client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")

# Define function to generate story (same as original)
def generate_story(messages, max_tokens, temperature, top_p):
    formatted_prompt = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "system":
            formatted_prompt += f"<|system|>\n{content}</s>\n"
        elif role == "user":
            formatted_prompt += f"<|user|>\nWrite a story about: {content}</s>\n"
        elif role == "assistant":
            formatted_prompt += f"<|assistant|>\n{content}</s>\n"
    formatted_prompt += "<|assistant|>\n"
    return client.text_generation(
        formatted_prompt,
        max_new_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
    )

# Define response function (same as original)
def respond(message, chat_history, max_tokens, temperature, top_p):
    system_message = {
        "role": "system",
        "content": "You are an AI story generator. Always respond with creative stories based on user inputs.",
    }
    messages = [system_message] + chat_history + [{"role": "user", "content": message}]
    story = generate_story(messages, max_tokens, temperature, top_p)
    updated_history = chat_history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": story},
    ]
    return updated_history, ""

# Create and launch Gradio interface (Colab-optimized)
def run_colab_interface():
    demo = gr.Blocks()

    with demo:
        gr.Markdown("# 📖 AI Story Generator (Colab Version)")
        with gr.Row():
            chatbot = gr.Chatbot(
                type="messages",
                label="Story Session",
                height=500,
                container=False
            )

        with gr.Row():
            with gr.Column(scale=4):
                user_input = gr.Textbox(
                    label="Your Story Prompt",
                    placeholder="Enter your story idea...",
                    container=False
                )
                submit_btn = gr.Button("Generate", variant="primary")

            with gr.Column(scale=1):
                max_tokens = gr.Slider(128, 2048, 512, step=64, label="Story Length")
                temperature = gr.Slider(0.1, 1.5, 0.7, step=0.1, label="Creativity")
                top_p = gr.Slider(0.1, 1.0, 0.95, step=0.05, label="Focus")
                clear_btn = gr.Button("Clear History", variant="secondary")

        # Event handlers
        submit_btn.click(
            respond,
            [user_input, chatbot, max_tokens, temperature, top_p],
            [chatbot, user_input]
        )
        user_input.submit(
            respond,
            [user_input, chatbot, max_tokens, temperature, top_p],
            [chatbot, user_input]
        )
        clear_btn.click(
            lambda: [], None, chatbot, queue=False
        )

    # Colab-specific launch configuration
    demo.launch(debug=True, share=True, height=800)

# Run the interface
if __name__ == "__main__":
    run_colab_interface()

Enter your Hugging Face token: ··········
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0e2e2792195a19a6fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
